In [1]:
#import libraries
import pandas as pd
import numpy as np
import sqlite3 as sql
!python -m pip install -U pip
!python -m pip install -U matplotlib
!pip install -U textblob
!python -m textblob.download_corpora
!pip install -U scikit-learn
!pip install seaborn

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:


ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'c:\\python310\\lib\\site-packages\\pip-21.2.4.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\fabia\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
#load dataset
reviews_df = pd.read_csv('C:\\Users\\fabia\OneDrive\Desktop\ZHAW\thouefab-Vorstudie\Womens Clothing E-Commerce Reviews.csv', index_col=0)
reviews_df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
#reviews_df = reviews_df.sample(10000)
# reviews_df = reviews_df[:10]

In [4]:
reviews_df["Review Text"].fillna("neutral", inplace= True)

In [5]:
reviews_df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")
reviews_df["Review Text"] = reviews_df["Review Text"].apply(nlp)

In [7]:
topics = []
i = 1
for index, row in reviews_df.iterrows():
    for sentence in row["Review Text"].sents:
        sentenceDocument = nlp(sentence.text)
        descriptiveTerm = ""
        target = ""
        for token in sentenceDocument:
            if token.dep_ == "nsubj" and token.pos_ == "NOUN":
                target = token.text
            if token.pos_ == "ADJ":
                prepend = ""
                for child in token.children:
                    if (child.pos_ != "ADV"):
                        continue
                    prepend += child.text + " "
                descriptiveTerm = prepend + token.text
        if (target):    # (target and descriptiveTerm)
            topics.append({"topic": target, "description": sentence.text,  "label": row["Rating"], "document": i})  # descriptiveTerm vs. sentence.text
   
    i = i + 1
# just for output
print(topics[:3])

[{'topic': 'half', 'description': 'overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers.', 'label': 3, 'document': 3}, {'topic': 'flaw', 'description': 'imo, a major design flaw was the net over layer sewn directly into the zipper - it c', 'label': 3, 'document': 3}, {'topic': 'shirt', 'description': 'This shirt is very flattering to all due to the adjustable front tie.', 'label': 5, 'document': 5}]


In [8]:
from textblob import TextBlob
for topic in topics:
    sentiment = TextBlob(topic["description"]).sentiment
    topic["textblob"] = {"polarity": sentiment.polarity, "subjectivity": sentiment.subjectivity} 

# just for output
print(topics[:3])

[{'topic': 'half', 'description': 'overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers.', 'label': 3, 'document': 3, 'textblob': {'polarity': 0.15768398268398268, 'subjectivity': 0.37316017316017314}}, {'topic': 'flaw', 'description': 'imo, a major design flaw was the net over layer sewn directly into the zipper - it c', 'label': 3, 'document': 3, 'textblob': {'polarity': 0.05416666666666667, 'subjectivity': 0.3}}, {'topic': 'shirt', 'description': 'This shirt is very flattering to all due to the adjustable front tie.', 'label': 5, 'document': 5, 'textblob': {'polarity': 0.037500000000000006, 'subjectivity': 0.3375}}]


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
for topic in topics:
    topic["sia"] = sia.polarity_scores(topic["description"])
     


# just for output
print(topics[:3])

[{'topic': 'half', 'description': 'overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers.', 'label': 3, 'document': 3, 'textblob': {'polarity': 0.15768398268398268, 'subjectivity': 0.37316017316017314}, 'sia': {'neg': 0.0, 'neu': 0.752, 'pos': 0.248, 'compound': 0.6428}}, {'topic': 'flaw', 'description': 'imo, a major design flaw was the net over layer sewn directly into the zipper - it c', 'label': 3, 'document': 3, 'textblob': {'polarity': 0.05416666666666667, 'subjectivity': 0.3}, 'sia': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}}, {'topic': 'shirt', 'description': 'This shirt is very flattering to all due to the adjustable front tie.', 'label': 5, 'document': 5, 'textblob': {'polarity': 0.037500000000000006, 'subjectivity': 0.3375}, 'sia': {'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'compound': 0.3804}}]


In [10]:
def flatten(topic):
    return (topic["label"], topic["document"], topic["topic"], topic["description"], topic["textblob"]["polarity"], 
            topic["textblob"]["subjectivity"], topic["sia"]["neg"], topic["sia"]["neu"], topic["sia"]["pos"], topic["sia"]["compound"])

In [11]:
import sqlite3

connection = None
try:
    connection = sqlite3.connect("vorstudie.db")
except Error as e:
    print(e)

insertStatement = "INSERT INTO review (label, document, topic, description, textblob_polarity, textblob_subjectivity, sia_neg, sia_neu, sia_pos, sia_compound ) VALUES (?,?,?,?,?,?,?,?,?,?)"
cursor = connection.cursor()

for topic in topics:
    cursor.execute(insertStatement, flatten(topic))

connection.commit()